First, import needed modules

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [21]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [22]:
btw = pd.read_csv(f'{path}/data/btw_treat.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
btw = btw[btw["Vote type"] == 'office']
btw = btw[btw["year"] >= 2013]

Split data in Erst- and Zweitstimme

In [23]:
erst = btw[btw['first_vote'] == 1]
zweit = btw[btw['second_vote'] == 1]

In [24]:
erst.columns

Index(['AGS', 'GEN', 'year', 'Land', 'post_2005', 'post_2009', 'post_2013',
       'post_2017', 'first_vote', 'second_vote', 'Vote type', 'Wähler',
       'Gültig', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere',
       'treatment', 'treated_0', 'treated_10', 'treated_20', 'treated_30',
       'treated_50', 'treated_100'],
      dtype='object')

Statsmodels

In [25]:
covar_names = {'treated_0:post_2017': 'treated_0*post_2017', 'treated_10:post_2017': 'treated_10*post_2017', 'treated_20:post_2017': 'treated_20*post_2017', 'treated_30:post_2017': 'treated_30*post_2017', 'treated_50:post_2017': 'treated_50*post_2017', 'treated_100:post_2017': 'treated_100*post_2017', }
covar_order =['post_2017', 'treated_0', 'treated_10', 'treated_20','treated_30', 'treated_50', 'treated_100', 'treated_0:post_2017', 'treated_10:post_2017', 'treated_20:post_2017', 'treated_30:post_2017', 'treated_50:post_2017', 'treated_100:post_2017']
treatment_dummies = '(treated_0 + treated_10 + treated_20 + treated_30 + treated_50 + treated_100)'

DiD for primary vote, treatment 50 in 2017

In [26]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ {treatment_dummies}*post_2017', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])}, missing='drop')
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.significant_digits(3)
stargazer.covariate_order(covar_order)
stargazer.rename_covariates(covar_names)
stargazer.show_model_numbers(False)
stargazer.add_custom_notes(['First note', 'Second note'])
stargazer.add_line('State dummies', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', ])
with open(f'{path}/tables/btw/erst_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

In [27]:
print(SPD.summary())

                            OLS Regression Results                            
Dep. Variable:                    SPD   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     34.98
Date:                Fri, 12 Aug 2022   Prob (F-statistic):           5.31e-88
Time:                        14:35:16   Log-Likelihood:                -82092.
No. Observations:               22130   AIC:                         1.642e+05
Df Residuals:                   22116   BIC:                         1.643e+05
Df Model:                          13                                         
Covariance Type:              cluster                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                22.74

DiD for secondary vote, treatment 50 in 2017

In [28]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ ({treatment_dummies})*post_2017 + Land', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) #np.array(zweit[['AGS', 'year']])
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.significant_digits(3)
stargazer.covariate_order(covar_order)
stargazer.rename_covariates(covar_names)
stargazer.show_model_numbers(False)
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Placebo DiD for treatments in 2005, 2009, 2013

Drop all observations after the treatment

In [29]:
erst_pl = erst[erst['year'] <= 2013]
zweit_pl = zweit[zweit['year'] <= 2013]

Primary vote, post 2005

In [30]:
time_dummies = '(post_2005 + post_2009 + post_2013)'

In [31]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ {treatment_dummies}*{time_dummies}', data=erst_pl).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst_pl[['AGS']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/erst_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 27, but rank is 22
  warnings.warn('covariance of constraints does not have full '
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 27, but rank is 22
  warnings.warn('covariance of constraints does not have full '
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 27, but rank is 22
  warnings.warn('covariance of constraints does not have full '
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance 

Secondary vote, post 2005

In [32]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2005 + Land', data=zweit_pl).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit_pl[['AGS', 'year']])}) # zweit['AGS']
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

ZeroDivisionError: float division by zero